In [1]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
import os
from matplotlib import colors
from random import choice
from experiment_class import Experiment,clean_exp_with_hyphaes
from hyphae_id import clean_and_relabel, get_mother, save_hyphaes

In [62]:
plate = 130
exp_clean = pickle.load( open( f'Data/'+f"experiment_{plate}.pick", "rb" ) )

In [63]:
def resolve_ambiguity_two_ends(hyphaes,bottom_threshold=0.98):
    root_hyph={}
    hyphae_two_ends=[hyph for hyph in hyphaes if hyph.root.degree(hyph.ts[0])==1]
    print(len(hyphae_two_ends))
    to_remove = []
    x_boundaries = hyphaes[0].experiment.boundaries_x
    y_boundaries = hyphaes[0].experiment.boundaries_y
    for hyph in hyphae_two_ends:
        t0 = hyph.ts[0]
        if not hyph.root.pos(t0)[0]>=bottom_threshold*x_boundaries[1]:
            nodes,edges  = hyph.get_nodes_within(t0)
            mini = np.inf
            found = False
            for i,edge in enumerate(edges):
                if edge.end.degree(t0) >=4:
                    if edge.end.degree(t0)>=5:
                        print(hyph,edge.end, hyph.ts)
                    else:
                        next_edge = edges[i+1]
                        angle = np.cos((edge.orientation_end(t0,50)-next_edge.orientation_begin(t0,50))/360*2*np.pi)
                        if angle<mini:
                            found= True
                            maxi=angle
                            root_candidate = edge.end
            if found:
                root_hyph[hyph] = root_candidate
    ends = {hyph.end : hyph for hyph in hyphaes}
    for hyph in root_hyph.keys():
        if hyph.root in ends:
            ends[hyph.root].root = root_hyph[hyph]
        hyph.root = root_hyph[hyph]
    for hyph in hyphaes[0].experiment.hyphaes:
        hyph.update_ts()
    return(root_hyph)

In [88]:
from experiment_class import Node
def solve_degree4(exp):
    hyphae_with_degree4 = {}
    exp_clean= clean_exp_with_hyphaes(exp)
    nx_graph_cleans = [nx.Graph.copy(nx_g) for nx_g in exp.nx_graph]
    exp_clean.nx_graph = nx_graph_cleans
    for hyph in exp.hyphaes:
        t0 = hyph.ts[-1]
        nodes,edges = hyph.get_nodes_within(t0)
        hyphae_with_degree4[hyph]=[]
        for node in nodes:
            if exp.get_node(node).degree(t0)>=4:
                hyphae_with_degree4[hyph].append(exp.get_node(node))
    roots = [hyph.root for hyph in exp.hyphaes]
    ends = [hyph.end for hyph in exp.hyphaes]
    solved_node = []
    solved = []
    for hyph in hyphae_with_degree4.keys():
        for node in hyphae_with_degree4[hyph]:
            if node not in roots and node not in ends and node not in solved_node:
                for t in node.ts():
                    solved.append((t,node.neighbours(t)))
                    solved_node.append(node)
                    if node.degree(t)==4:
                        pairs = []
                        for edge in node.edges(t):
                            mini=np.inf
                            for edge_candidate in node.edges(t):
                                angle=np.cos((edge.orientation_begin(t,100)-edge_candidate.orientation_begin(t,100))/360*2*np.pi)
                                if angle<mini:
                                    winner = edge_candidate
                                    mini=angle
                            if (edge,winner) not in pairs and (winner, edge) not in pairs:
                                pairs.append((edge,winner))
                        for pair in pairs:
                            right_n = pair[0].end
                            left_n = pair[1].end
                            right_edge = pair[0].pixel_list(t)
                            left_edge = list(reversed(pair[1].pixel_list(t)))
                            pixel_list = left_edge+right_edge[1:]
                            info={'weight':len(pixel_list),'pixel_list':pixel_list}
                            if right_n!=left_n:
                                exp_clean.nx_graph[t].add_edges_from([(left_n.label,right_n.label,info)])
                        exp_clean.nx_graph[t].remove_node(node.label)
                        if len(list(nx.connected_components(exp_clean.nx_graph[t])))>=2:
                            print(pairs)
                            print(t,node,len(list(nx.connected_components(exp_clean.nx_graph[t]))))
    exp_clean.nodes = []
    labels = {int(node) for g in exp_clean.nx_graph for node in g}
    for label in labels:
        exp_clean.nodes.append(Node(label,exp_clean))
    exp_clean_relabeled= clean_exp_with_hyphaes(exp_clean)
    print(len(solved))
    return(exp_clean_relabeled,solved)

In [68]:
solve_two_ends = resolve_ambiguity_two_ends(exp_clean.hyphaes)

79
(Node(3679), Node(2354)) 2348 [4]
(Node(5931), Node(4056)) 5930 [12]
(Node(8897), Node(6619)) 8378 [21, 22]


In [ ]:
[(Edge(207,143), Edge(207,227)), (Edge(207,185), Edge(207,217))]

In [ ]:
exp_new,solved = solve_degree4(exp_clean)

In [80]:
plt.close('all')
exp_clean.get_node(207).ts()

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

In [85]:
exp_clean.plot([4,8,12],[[207,143,227,185,217]]*3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
exp_clean.plot([0,10,20],[[118,144,146,144,134,144,155]]*3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …